In [179]:
import pandas as pd
import os
from os import listdir
from hashlib import blake2b
from pathlib import Path
import numpy as np
import string
import glob

In [180]:
# Concatenate all files 
df = pd.concat([pd.read_csv(file) for file in glob.glob('/home/jovyan/data-vol-1/ODHF/LODE-ECDO/scripts/HealthFacilities/V2/2-OpenTabulate/opentabulate/data/output/*.csv')])


In [181]:
# Removed "idx" (because redundant) and 'active' (because only one dataset had this info, and would be hard to keep updated). 
# **** DON'T FORGET TO ADD PROVINCE LATER ****

df = df[['facility_name', 'facility_type', 'health_authority', 'alternative_name', 'number_beds',
'address_str', 'street_no', 'street_name', 'street_addr', 'city', 'province', 'postal_code', 'provider', 
'latitude', 'longitude', 'phone', 'email', 'website' ]]

In [185]:


# Make postal codes consistent and put spaces instead of '_' + capitalize
df['postal_code'] = df['postal_code'].str.replace(' ','').str.upper()
df['facility_type'] = df['facility_type'].str.replace('_',' ')

# The code below is to convert the columns to integers. I've only needed to use it once and subsequent tries spit out an error. 
# But if you find these columns become floats again, uncomment and run once 
df['number_beds'] = df['number_beds'].round(0).astype(pd.Int64Dtype())
df['street_no'] = df['street_no'].round(0).astype(pd.Int64Dtype())

In [186]:
# Standardize facility_type
pharkeywords = 'pharmacy|pharmacies'
hoskeywords = 'hospital|hôpital|hôpitaux|cancer'
ltckeywords = 'residence|residential|résidence|nursing|palliative|senior|special care|seinor|long term|hospice|C.H.S.L.D.'
ambkeywords = 'community|clinic|walk in|primary care|tertiary|C.L.S.C.|regional'
covidkeywords = 'covid|immunization|vaccine'

df.loc[df['facility_type'].str.contains(pharkeywords, case=False, na=False), 'facility_type'] = 'TYPE 0'
df.loc[df['facility_type'].str.contains(hoskeywords, case=False, na=False), 'facility_type'] = 'TYPE 1'
df.loc[df['facility_type'].str.contains(ltckeywords, case=False, na=False), 'facility_type'] = 'TYPE 2'
df.loc[df['facility_type'].str.contains(ambkeywords, case=False, na=False), 'facility_type'] = 'TYPE 3'
df.loc[df['facility_type'].str.contains(covidkeywords, case=False, na=False), 'facility_type'] = 'TYPE 4'

In [187]:
df = df.reset_index(drop=True)
df.to_csv('merged.csv')
df.to_csv('/home/jovyan/data-vol-1/ODHF/LODE-ECDO/scripts/HealthFacilities/V2/4-Parsing/merged.csv')
print(df['facility_type'].unique())
#list_street_no = df['street_no'].unique()
#print(list_street_no)

df.dtypes

[nan 'TYPE 1' 'Health centre' 'TYPE 2' 'TYPE 3' 'TYPE 0'
 'Environmental Health' 'Rehabilitation' 'TYPE 4'
 'Specialized Care Bed Homes' 'West Prince Health Network'
 'Assisted Living' 'Children and Youth Mental Health'
 'Family and Children Services' 'Family Health Teams' 'Mental Health'
 'Public Health' 'Family Health Team'
 'Autres services médicaux et de santé' 'AIDS Bureau'
 'Independent Health Facility' 'Family Health Team - Contract'
 'Laboratory - Specimen Collection Centre' "Children's Treatment Centre"
 'Mental Health and Addiction Organization' 'Long-Term Care Home'
 'Retirement Home' 'Public Health Unit Office' 'Chronic Care'
 'Psychiatric' 'Urgent Care' 'Long care home' 'Basic Medical Needs'
 'Healthcare and emergency services' 'Primary Health Care to'
 'Acquired Injury' 'personal care homes' 'Other Organization'
 'Emergency Room Care' 'Urgent Care Centres']


facility_name        object
facility_type        object
health_authority     object
alternative_name     object
number_beds           Int64
address_str          object
street_no             Int64
street_name          object
street_addr          object
city                 object
province             object
postal_code          object
provider             object
latitude            float64
longitude           float64
phone                object
email                object
website              object
dtype: object

In [101]:
def make_temp_col(df):
    df_temp=df.copy()
    cols=['source_id','facility_name','address_str','provider']
    del_list=[" ","-","'","."]
    for col in cols:
    
        df_temp[col]=df_temp[col].str.upper()
        df_temp[col]=df_temp[col].fillna('NULL')
    
        for i in del_list:
            df_temp[col]=df_temp[col].str.replace(i,'',regex=False)
    df_temp['temp']=df_temp['source_id']+'-'+df_temp['facility_name']+'-'+df_temp['address_str']+'-'+df_temp['provider']
    return df_temp['temp']

df['temp']=make_temp_col(df)
df['idx']=df['temp'].apply(GetHash)

KeyError: 'source_id'

In [ ]:
df.loc[~df.latitude.isnull(), 'geo_source']='Source'

print(len(df), 'entries in database')

df.to_csv('ODHFv2_concat.csv',index=False)
